In [ ]:
using DatasetManager, LabDataSources, C3D, DataFrames, ParkinsonsDualTaskCoordination, Statistics, Peaks, Biomechanics, PlotlyJS, HypothesisTests

In [ ]:
rootdir = # REDACTED

rawpath = joinpath(rootdir, "data", "parkinsons", "raw")

subsets = [
    DataSubset("c3d", Source{C3DFile}, joinpath(rawpath, "PD_ARMS_STAB"), "Subject */_/*.c3d"),
    DataSubset("events", V3DEventsSource, joinpath(rawpath, "events"), "subject-[0-9]*/*.tsv")
]
labels = Dict(:arms => [ ["NONE", "NA"] => "held", ["AS", "Norm", "NORM", "normal"] => "norm"],
                 :kind => [ ["(?<=NONE|NORM|held|norm)S", "BA", "single(?!task)"] => "singletask", ["(?<=NONE|NORM|norm|held)C", "CO", "CP", "dual(?!task)"] => "dualtask",
                           "PO" => "pert", ["PARK", "(?<=_)TR(?=_)"] => "park" ],
                 :pert_side => [ ["[Rr](?=[ST]|(?i:slip|trip))", "RIGHT"] => "right", ["[Ll](?=[ST]|(?i:slip|trip))", "LEFT"] => "left"],
                 :pert_type => ["NP" => "steadystate", "(?i:(?<=[RL]|right|left))T" => "trip", "(?i:(?<=[RL]|right|left))S" => "slip"])
conds = TrialConditions((:arms,:kind,:pert_side,:pert_type), labels; required=(:arms,:kind),
    subject_fmt=r"(?<=Subject |N|subject-)(?<subject>\d+)",
    defaults=Dict(:pert_type => "steadystate", :pert_side => "NA"))

# Read all perturbations
trials = findtrials(subsets, conds;
     ignorefiles = [
        joinpath(rawpath, "PD_ARMS_STAB", "Subject 08/_/held_single_ltrip.c3d"),
    ])
filter!(trials) do trial
    hassource(trial, "events")
end
summarize(trials)

In [ ]:
function evaluatepredictions(alg, trial)
    seg = Segment(trial, "events")
    segr = SegmentResult(seg)
    res = results(segr)
    
    events = readsegment(seg)
    pred = alg(trial)
    @assert keys(pred) ⊆ keys(events)
    
    for event in keys(pred)
        _, _, dists, missed = matchevents(pred[event], events[event])
        res[event*"_err"] = dists
        res[event*"_miss"] = missed
        
        ev_diff = diff(pred[event])
        ev_qtl = quantile(ev_diff, [0.25,0.5,0.75])
        res[event*"_outlyr"] = filter(>(ev_qtl[2]+ev_qtl[3]-ev_qtl[1])∘abs, ev_diff .- ev_qtl[2])
    end
    res["FC_err"] = vcat(res["LFC_err"], res["RFC_err"])
    res["FO_err"] = vcat(res["LFO_err"], res["RFO_err"])
    res["FC_miss"] = vcat(res["LFC_miss"], res["RFC_miss"])
    res["FO_miss"] = vcat(res["LFO_miss"], res["RFO_miss"])
    
    return segr
end

In [ ]:
srs = analyzedataset(trials, V3DEventsSource) do trial
    evaluatepredictions(ParkinsonsDualTaskCoordination.roerdink2008, trial)
end;
filter!(!isempty∘results, srs);

In [ ]:
fc_errs = reduce(vcat, getindex.(results.(srs), "FC_err"))
fo_errs = reduce(vcat, getindex.(results.(srs), "FO_err"));

In [ ]:
plot([
        histogram(;x=fc_errs, xbins_size=.01, histnorm="probability", name="Heelstrike"),
        histogram(;x=fo_errs, xbins_size=.01, histnorm="probability", name="Toe-off"),
    ],
    Layout(;
        xaxis_dtick=.01,
        xaxis_range=[-.1,.1],
        height=350,
        width=800,
); config=PlotConfig(;scrollZoom=false))

In [ ]:
fc_test = OneSampleTTest(fc_errs)

In [ ]:
fo_test = OneSampleTTest(fo_errs)

In [ ]:
outliers = reduce(vcat, filter(!isempty, getindex.(results.(srs), "LFO_outlyr")))
length(outliers), mean(outliers), extrema(outliers)